# Content Change Detection

Detects content changes using chunk-based comparison with version tracking.

**✨ Now supports both SQLite (local development) and Databricks Unity Catalog (production)!**

The notebook automatically detects your environment and configures accordingly:
- **Local Environment**: Uses SQLite database files
- **Databricks Environment**: Uses Unity Catalog Delta tables with Spark operations

## Overview

1. Load modified content from `content_repo` (ingested via [3_content_repo_edit.ipynb](3_content_repo_edit.ipynb))
2. **Chunk modified content and ingest into `content_chunks`** (happens in this notebook)
3. Load baseline chunks from `content_chunks` (v1 files)
4. Load current chunks from `content_chunks` (recently modified files)
5. Detect changes (NEW/MODIFIED/DELETED/UNCHANGED) by comparing checksums
6. Store results in `content_change_log`

## Architecture

- **Table**: `content_chunks` (NOT `content_checksums`)
- **Class**: `ChunkIngestion` (NOT `ChecksumIngestion`)
- **Relationship**: Chunks are linked to `content_repo` via FK `ud_source_file_id`
- **Versioning**: Compare v1 baseline vs v2+ current using `raw_file_version_nbr`
- **Database Support**: 
  - SQLite for local development
  - Databricks Unity Catalog Delta tables for production

## Scope

This notebook handles:
- **Chunking**: Breaking modified content into fixed-size chunks
- **Checksum Computation**: Computing SHA-256 checksums for each chunk
- **Storage**: Ingesting chunks into `content_chunks` with FK relationships
- **Change Detection**: Comparing baseline (v1) vs current (v2+) chunks
- **Result Storage**: Recording detected changes

## Configuration

Set `SINCE_DATE` below to filter content modified after a specific date.

In [1]:
# DELETE ALL ENTRIES (UNCOMMENT TO CLEAR DATA)
# 
# if USE_UNITY_CATALOG:
#     # Unity Catalog: Delete from Delta table
#     spark.sql(f"DELETE FROM {CATALOG_NAME}.{SCHEMA_NAME}.content_change_log")
#     
#     # Verify
#     count_df = spark.sql(f"SELECT COUNT(*) as count FROM {CATALOG_NAME}.{SCHEMA_NAME}.content_change_log")
#     count = count_df.collect()[0]['count']
#     print(f"Remaining records: {count}")
# 
# else:
#     # SQLite: Original code
#     import sqlite3
#     conn = sqlite3.connect(DB_PATH)
#     conn.execute("DELETE FROM content_change_log")
#     conn.commit()
#     cursor = conn.execute("SELECT COUNT(*) FROM content_change_log")
#     count = cursor.fetchone()[0]
#     print(f"Remaining records: {count}")
#     conn.close()

---

## Step 1: Setup and Configuration

In [2]:
# CONFIGURATION - Edit these values as needed

from utils.env_utils import is_databricks

if is_databricks():
    # Databricks: Setup widgets
    dbutils.widgets.text("catalog", "onedata_us_east_1_shared_dit", "Catalog")
    dbutils.widgets.text("schema", "ssot_raw_ditagen_dit", "Schema")
    dbutils.widgets.text("volume", "datacloud-dita-gen-volume", "Volume")
    
    CATALOG = dbutils.widgets.get("catalog")
    SCHEMA = dbutils.widgets.get("schema")
    VOLUME = dbutils.widgets.get("volume")
    
    # Databricks paths
    PROJECT_ROOT = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/FAQ_update"
    DATA_DIR = f"{PROJECT_ROOT}/data_ingestion/data"
    MARKDOWN_DIR = f"{DATA_DIR}/markdown"
    
    print(f"Databricks mode: {CATALOG}.{SCHEMA}")
    print(f"Project root: {PROJECT_ROOT}")
else:
    # Local environment
    CATALOG = None
    SCHEMA = None
    VOLUME = None
    
    # Local paths (relative to notebook location)
    import os
    from pathlib import Path
    NOTEBOOK_DIR = Path(os.getcwd())
    # When running from faq_update directory, PROJECT_ROOT should be faq_update itself
    PROJECT_ROOT = NOTEBOOK_DIR if NOTEBOOK_DIR.name == "faq_update" else NOTEBOOK_DIR
    DATABASE_DIR = PROJECT_ROOT / "databases"
    DB_PATH = str(DATABASE_DIR / "faq_update.db")  # Convert to string for sqlite3.connect()
    DATA_DIR = PROJECT_ROOT / "data_ingestion" / "data"
    MARKDOWN_DIR = DATA_DIR / "markdown"
    
    print(f"Local mode")
    print(f"Database: {DB_PATH}")
    print(f"Project root: {PROJECT_ROOT}")

ModuleNotFoundError: No module named 'utils'

In [ ]:
import sys
import os
import sqlite3
import pandas as pd
from pathlib import Path
from datetime import datetime
import hashlib
import uuid
from typing import Dict, List, Optional

# Add PROJECT_ROOT to path so we can import faq_update packages
if isinstance(PROJECT_ROOT, Path):
    sys.path.insert(0, str(PROJECT_ROOT))
else:
    sys.path.insert(0, PROJECT_ROOT)

print("=" * 100)
print("🔍 CONTENT CHANGE DETECTION")
print("=" * 100)
print(f"📁 Project Root: {PROJECT_ROOT}")

if is_databricks():
    print(f"📁 Database: Unity Catalog managed tables ({CATALOG}.{SCHEMA})")
    print(f"📁 Mode: Databricks with Unity Catalog")
else:
    print(f"📁 Database: {DB_PATH}")
    print(f"📁 Database exists: {os.path.exists(DB_PATH)}")
    print(f"📁 Mode: Local SQLite")

print("=" * 100)

🔍 CONTENT CHANGE DETECTION
📁 Project Root: c:\Users\praka\projects\FAQ_combined\faq_update
📁 Database: c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db
📁 Database exists: True
📁 Mode: Local SQLite


### Import Detection Modules and Chunking

In [ ]:
from detection.checksum_extractor import ChecksumExtractor
from detection.content_change_detector import ContentChangeDetector
from detection.database_queries import ChangeDetectionQueries
from database.models import ChangeType, ContentChange
from chunking import SimpleChunker
from typing import List

# Always import ChunkIngestion - it's needed for both modes
from data_ingestion import ChunkIngestion

print("✅ Detection modules imported")

if is_databricks():
    print("✅ Databricks mode - using Spark for database operations")
else:
    print("✅ Local SQLite mode - using traditional database connections")

✅ Detection modules imported
✅ Local SQLite mode - using traditional database connections


### Configure Detection Parameters

In [ ]:
SINCE_DATE = "2025-11-02 10:20:27"
SIMILARITY_THRESHOLD = 0.8
CHUNK_SIZE = 1000  # Same as in notebook 1
DETECTION_RUN_ID = f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:8]}"

print(f"\nConfiguration:")
print(f"  Since Date: {SINCE_DATE}")
print(f"  Chunk Size: {CHUNK_SIZE} characters")
print(f"  Similarity Threshold: {SIMILARITY_THRESHOLD}")
print(f"  Run ID: {DETECTION_RUN_ID}")
print(f"\n{'='*80}")


Configuration:
  Since Date: 2025-10-27 17:41:10
  Chunk Size: 1000 characters
  Similarity Threshold: 0.8
  Run ID: run_20251030_232231_62f369ff



In [ ]:
# Unity Catalog Database Adapter for Change Detection
class DatabricksChangeDetection:
    """
    Unity Catalog version of change detection operations using Spark/Delta tables.
    """
    
    def __init__(self, catalog_name: str, schema_name: str):
        """Initialize with Unity Catalog coordinates."""
        self.catalog_name = catalog_name
        self.schema_name = schema_name
        self.content_repo_table = f"{catalog_name}.{schema_name}.content_repo"
        self.content_chunks_table = f"{catalog_name}.{schema_name}.content_chunks"
        self.content_change_log_table = f"{catalog_name}.{schema_name}.content_change_log"
        
    def load_modified_content(self, since_date: str) -> pd.DataFrame:
        """Load modified content from Unity Catalog Delta table."""
        query = f"""
        SELECT *
        FROM {self.content_repo_table}
        WHERE last_modified_dt > '{since_date}' AND file_status = 'Active'
        ORDER BY last_modified_dt DESC
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()
    
    def load_all_content(self) -> pd.DataFrame:
        """Load all active content for comparison."""
        query = f"""
        SELECT ud_source_file_id, raw_file_nme, last_modified_dt, file_status
        FROM {self.content_repo_table}
        WHERE file_status = 'Active'
        ORDER BY last_modified_dt DESC
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()
    
    def load_baseline_chunks(self) -> pd.DataFrame:
        """Load baseline chunks (version 1 files) from Unity Catalog."""
        query = f"""
        SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
        FROM {self.content_chunks_table} cc
        JOIN {self.content_repo_table} cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.raw_file_version_nbr = 1
          AND cc.status = 'active'
          AND cr.file_status = 'Active'
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()
    
    def load_current_chunks(self, since_date: str) -> pd.DataFrame:
        """Load current chunks (recently modified files) from Unity Catalog."""
        query = f"""
        SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
        FROM {self.content_chunks_table} cc
        JOIN {self.content_repo_table} cr ON cc.ud_source_file_id = cr.ud_source_file_id
        WHERE cr.last_modified_dt > '{since_date}'
          AND cc.status = 'active'
          AND cr.file_status = 'Active'
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()
    
    def store_changes(self, changes: List, detection_run_id: str) -> int:
        """Store change detection results to Unity Catalog Delta table."""
        if not changes:
            print("No changes to store")
            return 0
        
        from pyspark.sql import Row
        from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, BooleanType, TimestampType
        from datetime import datetime
        
        # Define explicit schema to match content_change_log table (excluding auto-generated change_id)
        schema = StructType([
            StructField("content_checksum", StringType(), False),
            StructField("previous_checksum", StringType(), True),
            StructField("file_name", StringType(), False),
            StructField("requires_faq_regeneration", BooleanType(), False),
            StructField("change_type", StringType(), True),
            StructField("similarity_score", DoubleType(), True),
            StructField("similarity_method", StringType(), True),
            StructField("diff_data", StringType(), True),
            StructField("total_faqs_at_risk", IntegerType(), False),
            StructField("affected_question_count", IntegerType(), False),
            StructField("affected_answer_count", IntegerType(), False),
            StructField("detection_run_id", StringType(), False),
            StructField("detection_timestamp", TimestampType(), False)
        ])
        
        # Convert changes to rows with explicit types
        rows = []
        current_timestamp = datetime.now()
        
        for change in changes:
            requires_regen = change.change_type in (
                "new_content", "modified_content", "deleted_content"
            )
            
            rows.append(Row(
                content_checksum=str(change.new_checksum),
                previous_checksum=str(change.old_checksum) if change.old_checksum else None,
                file_name=str(change.file_name),
                requires_faq_regeneration=bool(requires_regen),
                change_type=str(change.change_type.value),
                similarity_score=float(change.similarity_score) if change.similarity_score is not None else None,
                similarity_method="hybrid",
                diff_data=str(change.llm_friendly_diff) if change.llm_friendly_diff else None,
                total_faqs_at_risk=int(0),
                affected_question_count=int(0),
                affected_answer_count=int(0),
                detection_run_id=str(detection_run_id),
                detection_timestamp=current_timestamp
            ))
        
        # Create DataFrame with explicit schema
        df = spark.createDataFrame(rows, schema)
        df.write.format("delta").mode("append").saveAsTable(self.content_change_log_table)
        
        print(f"💾 Stored {len(rows)} changes to Unity Catalog Delta table: {self.content_change_log_table}")
        return len(rows)
    
    def verify_results(self, detection_run_id: str) -> pd.DataFrame:
        """Verify stored results from Unity Catalog."""
        query = f"""
        SELECT change_type, COUNT(*) as count
        FROM {self.content_change_log_table} 
        WHERE detection_run_id = '{detection_run_id}'
        GROUP BY change_type
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()
    
    def get_all_results(self, detection_run_id: str) -> pd.DataFrame:
        """Get all results for a detection run from Unity Catalog."""
        query = f"""
        SELECT *
        FROM {self.content_change_log_table} 
        WHERE detection_run_id = '{detection_run_id}'
        """
        
        spark_df = spark.sql(query)
        return spark_df.toPandas()


# Initialize appropriate database handler based on environment
if is_databricks():
    print("🔧 Initializing Unity Catalog change detection...")
    try:
        db_handler = DatabricksChangeDetection(CATALOG, SCHEMA)
        print(f"✓ Unity Catalog change detection initialized")
        print(f"  📋 Tables: {CATALOG}.{SCHEMA}.[content_repo|content_chunks|content_change_log]")
    except Exception as e:
        print(f"❌ Error initializing Unity Catalog change detection: {e}")
        print("Make sure you're running in a Databricks environment with proper Unity Catalog access")
        raise
else:
    print("🔧 Using SQLite database operations...")
    db_handler = None  # Will use direct SQLite queries
    print(f"✓ SQLite mode initialized")
    print(f"  📋 Database: {DB_PATH}")

🔧 Using SQLite database operations...
✓ SQLite mode initialized
  📋 Database: c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db


In [ ]:
# Unity Catalog Chunk Ingestion Adapter  
class DatabricksChunkIngestion:
    """
    Unity Catalog version of ChunkIngestion using Spark/Delta tables.
    """
    
    def __init__(self, catalog_name: str, schema_name: str):
        """Initialize with Unity Catalog coordinates."""
        self.catalog_name = catalog_name
        self.schema_name = schema_name
        self.table_name = f"{catalog_name}.{schema_name}.content_chunks"
        
    def ingest_chunks_for_file(self, ud_source_file_id: int, chunks_with_checksums: List, clear_existing: bool = False) -> dict:
        """Ingest chunks for a file into Unity Catalog Delta table."""
        try:
            from pyspark.sql import Row
            from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, TimestampType
            from datetime import datetime
            
            if clear_existing:
                # Delete existing chunks for this file
                spark.sql(f"""
                DELETE FROM {self.table_name} 
                WHERE ud_source_file_id = {ud_source_file_id}
                """)
            
            # Define schema to match existing table structure (excluding auto-generated chunk_id)
            schema = StructType([
                StructField("ud_source_file_id", LongType(), False),
                StructField("chunk_index", IntegerType(), False),
                StructField("content_checksum", StringType(), False),
                StructField("chunk_text", StringType(), False),
                StructField("chunk_page_number", IntegerType(), True),
                StructField("chunk_start_char", IntegerType(), True),
                StructField("chunk_end_char", IntegerType(), True),
                StructField("status", StringType(), False)
            ])
            
            # Convert chunks to Spark rows (chunk_id is auto-generated, created_at has default)
            rows = []
            for i, (chunk_text, checksum) in enumerate(chunks_with_checksums):
                rows.append(Row(
                    ud_source_file_id=int(ud_source_file_id),
                    chunk_index=int(i),
                    content_checksum=str(checksum),
                    chunk_text=str(chunk_text),
                    chunk_page_number=None,  # Not available in our data
                    chunk_start_char=int(i * len(chunk_text)),
                    chunk_end_char=int((i + 1) * len(chunk_text)),
                    status="active"
                ))
            
            if rows:
                # Create DataFrame with exact schema match
                spark_df = spark.createDataFrame(rows, schema)
                
                # Write to Delta table - schema should match exactly now
                spark_df.write \
                    .format("delta") \
                    .mode("append") \
                    .saveAsTable(self.table_name)
            
            return {
                "success": True,
                "rows_inserted": len(rows),
                "message": f"Successfully inserted {len(rows)} chunks"
            }
            
        except Exception as e:
            return {
                "success": False,
                "rows_inserted": 0,
                "message": f"Error ingesting chunks: {str(e)}"
            }


# Initialize appropriate chunk ingestion based on environment
if is_databricks():
    print("🔧 Initializing Unity Catalog chunk ingestion...")
    try:
        chunk_ingestion = DatabricksChunkIngestion(CATALOG, SCHEMA)
        print(f"✓ Unity Catalog chunk ingestion initialized: {CATALOG}.{SCHEMA}.content_chunks")
    except Exception as e:
        print(f"❌ Error initializing Unity Catalog chunk ingestion: {e}")
        print("Make sure you're running in a Databricks environment with Spark available")
        raise
else:
    print("🔧 Initializing SQLite chunk ingestion...")
    try:
        chunk_ingestion = ChunkIngestion(str(DB_PATH))
        print(f"✓ SQLite chunk ingestion initialized: {DB_PATH}")
    except Exception as e:
        print(f"❌ Error initializing SQLite chunk ingestion: {e}")
        raise

🔧 Initializing SQLite chunk ingestion...
✓ SQLite chunk ingestion initialized: c:\Users\praka\projects\FAQ_combined\faq_update\databases\faq_update.db


---
## 🌟 Multi-Environment Support

This notebook supports **both local development and Databricks production environments**:

### Local Environment (SQLite)
- ✅ Uses SQLite database files for storage
- ✅ Direct file system access for markdown content  
- ✅ Traditional `sqlite3` connections and pandas operations
- ✅ Perfect for development, testing, and small datasets

### Databricks Environment (Unity Catalog)
- ✅ Uses Unity Catalog Delta tables for storage
- ✅ Spark SQL operations for scalability and performance
- ✅ Unity Catalog volume storage for files
- ✅ Perfect for production, large datasets, and collaborative environments

**The notebook automatically detects your environment and configures accordingly using `utils.env_utils.is_databricks()`!**

---

---

## Step 2: Load Modified Content

In [ ]:
# First, let's verify the data and datetime filtering
if is_databricks():
    # Unity Catalog operations
    all_records = db_handler.load_all_content()
    print(f"\nTotal Active records: {len(all_records)}")
    print(f"\nAll modification dates:")
    print(all_records[['ud_source_file_id', 'raw_file_nme', 'last_modified_dt']].to_string())
    
    # Filter with SINCE_DATE
    modified_content_df = db_handler.load_modified_content(SINCE_DATE)
    
else:
    # SQLite operations (original code)
    with sqlite3.connect(DB_PATH) as conn:
        # Check all records with their modification dates
        all_records = pd.read_sql_query("""
            SELECT ud_source_file_id, raw_file_nme, last_modified_dt, file_status
            FROM content_repo
            WHERE file_status = 'Active'
            ORDER BY last_modified_dt DESC
        """, conn)
        
        print(f"\nTotal Active records: {len(all_records)}")
        print(f"\nAll modification dates:")
        print(all_records[['ud_source_file_id', 'raw_file_nme', 'last_modified_dt']].to_string())
        
        # Now filter with SINCE_DATE
        modified_content_df = pd.read_sql_query("""
            SELECT *
            FROM content_repo
            WHERE last_modified_dt > ? AND file_status = 'Active'
            ORDER BY last_modified_dt DESC
        """, conn, params=[SINCE_DATE])

print(f"\n{'='*80}")
print(f"Filtered records (since {SINCE_DATE}): {len(modified_content_df)}")
print(f"{'='*80}")

if len(modified_content_df) > 0:
    print("\nFiltered data preview:")
    display(modified_content_df[['ud_source_file_id', 'raw_file_nme', 'raw_file_version_nbr', 'last_modified_dt']].head(10))
else:
    print("\n⚠️ No records found! This might be a datetime format issue.")
    print(f"   SINCE_DATE value: {SINCE_DATE}")
    print(f"   Latest record date: {all_records['last_modified_dt'].max() if len(all_records) > 0 else 'N/A'}")


Total Active records: 24

All modification dates:
    ud_source_file_id               raw_file_nme     last_modified_dt
0                  15      Employee_Handbook.pdf  2025-10-31 03:10:21
1                  16     IT_Security_Policy.pdf  2025-10-31 03:10:21
2                  17     Remote_Work_Policy.pdf  2025-10-31 03:10:21
3                  18          Travel_Policy.pdf  2025-10-31 03:10:21
4                  19         Parental_Leave.pdf  2025-10-31 03:10:21
5                  20       Workplace_Safety.pdf  2025-10-31 03:10:21
6                  21       Health_Insurance.pdf  2025-10-31 03:10:21
7                  22        Vacation_Policy.pdf  2025-10-31 03:10:21
8                  23           Email_Policy.pdf  2025-10-31 03:10:21
9                  24       Equipment_Return.pdf  2025-10-31 03:10:21
10                  1      Employee_Handbook.pdf  2025-10-31 03:09:19
11                  2     IT_Security_Policy.pdf  2025-10-31 03:09:19
12                  3     Remote_Work_P

ud_source_file_id            raw_file_nme  raw_file_version_nbr  \
0                 15   Employee_Handbook.pdf                     2   
1                 16  IT_Security_Policy.pdf                     2   
2                 17  Remote_Work_Policy.pdf                     2   
3                 18       Travel_Policy.pdf                     2   
4                 19      Parental_Leave.pdf                     2   
5                 20    Workplace_Safety.pdf                     1   
6                 21    Health_Insurance.pdf                     2   
7                 22     Vacation_Policy.pdf                     2   
8                 23        Email_Policy.pdf                     2   
9                 24    Equipment_Return.pdf                     1   

      last_modified_dt  
0  2025-10-31 03:10:21  
1  2025-10-31 03:10:21  
2  2025-10-31 03:10:21  
3  2025-10-31 03:10:21  
4  2025-10-31 03:10:21  
5  2025-10-31 03:10:21  
6  2025-10-31 03:10:21  
7  2025-10-31 03:10:21  
8  2025-10-31 03:10:21  
9  2025-10-31 03:10:21

---

## Step 3: Chunk Modified Content and Ingest to content_chunks

This step chunks modified content and stores it in the `content_chunks` table with FK links to `content_repo`:

1. Read markdown files for modified content
2. Chunk into fixed-size pieces (1000 chars)
3. Compute SHA-256 checksums for each chunk
4. **Insert into `content_chunks` table with `ud_source_file_id` FK** (NOT `content_checksums`)

This creates the "current state" chunks (v2) that will be compared against baseline chunks (v1) to detect changes.

In [ ]:
print(f"\n{'='*80}")
print("  CHUNKING AND CHECKSUM COMPUTATION")
print(f"{'='*80}\n")

# Initialize chunker 
chunker = SimpleChunker(chunk_size=CHUNK_SIZE)

# Process each modified file
files_processed = 0
total_chunks = 0

for idx, row in modified_content_df.iterrows():
    # The path in database is relative to PROJECT_ROOT
    relative_path = row['extracted_markdown_file_path']
    
    if is_databricks():
        # In Databricks, paths are already absolute from the volume
        markdown_path = Path(relative_path)
    else:
        # In local, construct path from PROJECT_ROOT
        markdown_path = PROJECT_ROOT / relative_path
    
    if not markdown_path.exists():
        print(f"⚠️  WARNING: File not found: {markdown_path}")
        continue
    
    # Read content
    try:
        content = markdown_path.read_text(encoding='utf-8')
    except Exception as e:
        print(f"✗ ERROR reading {markdown_path}: {e}")
        continue
    
    # Chunk and compute checksums
    chunks_with_checksums = chunker.chunk_with_checksums(content)
    
    # Ingest chunks for this file with FK to content_repo
    result = chunk_ingestion.ingest_chunks_for_file(
        ud_source_file_id=row['ud_source_file_id'],
        chunks_with_checksums=chunks_with_checksums,
        clear_existing=True
    )
    
    if result['success']:
        files_processed += 1
        total_chunks += result['rows_inserted']
        print(f"  ✓ {row['raw_file_nme']:40s} (v{row['raw_file_version_nbr']}) → {result['rows_inserted']:3d} chunks")
    else:
        print(f"  ✗ {row['raw_file_nme']:40s} - Error: {result['message']}")

print(f"\n✓ Processed {files_processed} files")

if is_databricks():
    print(f"✓ Ingested {total_chunks} chunks into Unity Catalog table: {CATALOG}.{SCHEMA}.content_chunks")
else:
    print(f"✓ Ingested {total_chunks} chunks into SQLite content_chunks table")

print(f"\n✅ Modified content chunked and stored in database")


  CHUNKING AND CHECKSUM COMPUTATION

  ✓ Employee_Handbook.pdf                    (v2) →   1 chunks
  ✓ IT_Security_Policy.pdf                   (v2) →   1 chunks
  ✓ Remote_Work_Policy.pdf                   (v2) →   1 chunks
  ✓ Travel_Policy.pdf                        (v2) →   1 chunks
  ✓ Parental_Leave.pdf                       (v2) →   1 chunks
⚠️  WARNING: File not found: c:\Users\praka\projects\FAQ_combined\faq_update\data_ingestion\data\markdown\Workplace_Safety_v1.md
  ✓ Health_Insurance.pdf                     (v2) →   1 chunks
⚠️  WARNING: File not found: c:\Users\praka\projects\FAQ_combined\faq_update\data_ingestion\data\markdown\Vacation_Policy_v2.md
  ✓ Email_Policy.pdf                         (v2) →   1 chunks
  ✓ Equipment_Return.pdf                     (v1) →   1 chunks
  ✓ Employee_Handbook.pdf                    (v1) →   1 chunks
  ✓ IT_Security_Policy.pdf                   (v1) →   1 chunks
  ✓ Remote_Work_Policy.pdf                   (v1) →   1 chunks
  ✓ Benefits

---

## Step 4: Load Baseline and Current Chunks, Then Detect Changes

Load chunks from `content_chunks` table (with JOIN to `content_repo` for version filtering):
- **Baseline**: Chunks from version 1 files (`raw_file_version_nbr = 1`)
- **Current**: Chunks from recently modified files (filtered by `last_modified_dt`)

Then detect changes by comparing checksums and content.

In [ ]:
# Load baseline chunks (version 1 files) and current chunks
if is_databricks():
    # Unity Catalog operations
    baseline_df = db_handler.load_baseline_chunks()
    current_df = db_handler.load_current_chunks(SINCE_DATE)
    
else:
    # SQLite operations (original code)
    with sqlite3.connect(DB_PATH) as conn:
        baseline_df = pd.read_sql_query("""
            SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
            FROM content_chunks cc
            JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
            WHERE cr.raw_file_version_nbr = 1
              AND cc.status = 'active'
              AND cr.file_status = 'Active'
        """, conn)
    
    with sqlite3.connect(DB_PATH) as conn:
        current_df = pd.read_sql_query("""
            SELECT cc.content_checksum, cc.chunk_text, cr.raw_file_nme
            FROM content_chunks cc
            JOIN content_repo cr ON cc.ud_source_file_id = cr.ud_source_file_id
            WHERE cr.last_modified_dt > ?
              AND cc.status = 'active'
              AND cr.file_status = 'Active'
        """, conn, params=[SINCE_DATE])

print(f"Loaded {len(baseline_df)} baseline chunks (v1)")
print(f"Loaded {len(current_df)} current chunks (modified since {SINCE_DATE})")

# Initialize detector
detector = ContentChangeDetector(similarity_threshold=SIMILARITY_THRESHOLD)

# Prepare data structures for detector
previous_data = {}
for _, row in baseline_df.iterrows():
    previous_data[row['content_checksum']] = {
        'content_text': row['chunk_text'],
        'file_name': row['raw_file_nme']
    }

current_data = {}
for _, row in current_df.iterrows():
    current_data[row['content_checksum']] = {
        'text': row['chunk_text'],
        'file_name': row['raw_file_nme']
    }

print(f"\nPrepared {len(previous_data)} baseline checksums")
print(f"Prepared {len(current_data)} current checksums")

# Detect changes per file
all_changes = []
for file_name in modified_content_df["raw_file_nme"].unique():
    curr = {k: v for k, v in current_data.items() if v["file_name"] == file_name}
    prev = {k: v for k, v in previous_data.items() if v.get("file_name") == file_name}
    
    print(f"\nProcessing {file_name}:")
    print(f"  Current chunks: {len(curr)}")
    print(f"  Previous chunks: {len(prev)}")
    
    changes = detector.detect_changes(file_name, curr, prev, DETECTION_RUN_ID)
    all_changes.extend(changes)
    print(f"  Changes detected: {len(changes)}")

print(f"\n{'='*80}")
print(f"Total changes detected: {len(all_changes)}")
print(f"{'='*80}")

2025-10-30 23:25:39.283 | INFO     | detection.content_change_detector:__init__:83 - ContentChangeDetector initialized: algorithm=sha256, similarity_threshold=0.8, compute_llm_diffs=True, similarity_weights={'jaccard': 0.4, 'difflib': 0.6}
2025-10-30 23:25:39.298 | INFO     | detection.content_change_detector:detect_changes:121 - 🔍 Detecting changes for: Employee_Handbook.pdf
2025-10-30 23:25:39.300 | INFO     | detection.content_change_detector:detect_changes:122 -    Detection run: run_20251030_232231_62f369ff
2025-10-30 23:25:39.301 | INFO     | detection.content_change_detector:detect_changes:133 - 📊 Checksum analysis for Employee_Handbook.pdf:
2025-10-30 23:25:39.302 | INFO     | detection.content_change_detector:detect_changes:134 -    Current:   2 checksums
2025-10-30 23:25:39.303 | INFO     | detection.content_change_detector:detect_changes:135 -    Previous:  1 checksums
2025-10-30 23:25:39.304 | INFO     | detection.content_change_detector:detect_changes:136 -    New:       1

Loaded 13 baseline chunks (v1)
Loaded 20 current chunks (modified since 2025-10-27 17:41:10)

Prepared 13 baseline checksums
Prepared 20 current checksums

Processing Employee_Handbook.pdf:
  Current chunks: 2
  Previous chunks: 1
  Changes detected: 2

Processing IT_Security_Policy.pdf:
  Current chunks: 2
  Previous chunks: 1
  Changes detected: 2

Processing Remote_Work_Policy.pdf:
  Current chunks: 2
  Previous chunks: 1
  Changes detected: 2

Processing Travel_Policy.pdf:
  Current chunks: 2
  Previous chunks: 1
  Changes detected: 2

Processing Parental_Leave.pdf:
  Current chunks: 1
  Previous chunks: 0
  Changes detected: 1

Processing Workplace_Safety.pdf:
  Current chunks: 0
  Previous chunks: 0
  Changes detected: 0

Processing Health_Insurance.pdf:
  Current chunks: 2
  Previous chunks: 1
  Changes detected: 2

Processing Vacation_Policy.pdf:
  Current chunks: 0
  Previous chunks: 0
  Changes detected: 0

Processing Email_Policy.pdf:
  Current chunks: 2
  Previous chunks: 1

2025-10-30 23:25:39.471 | INFO     | detection.content_change_detector:detect_changes:137 -    Deleted:   0 checksums
2025-10-30 23:25:39.477 | INFO     | detection.content_change_detector:detect_changes:138 -    Unchanged: 1 checksums
2025-10-30 23:25:39.482 | INFO     | detection.content_change_detector:detect_changes:164 - ⚡ No similarity matching needed (no overlap between new/deleted)
2025-10-30 23:25:39.498 | INFO     | detection.content_change_detector:detect_changes:226 - ✅ Changes detected: 1
2025-10-30 23:25:39.506 | INFO     | detection.content_change_detector:detect_changes:227 -    New:      0
2025-10-30 23:25:39.510 | INFO     | detection.content_change_detector:detect_changes:228 -    Modified: 0
2025-10-30 23:25:39.513 | INFO     | detection.content_change_detector:detect_changes:229 -    Deleted:  0
2025-10-30 23:25:39.515 | INFO     | detection.content_change_detector:detect_changes:230 -    Unchanged: 1
2025-10-30 23:25:39.516 | INFO     | detection.content_change_de

  Changes detected: 1

Processing Benefits_Guide.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Payroll_Processing.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Code_of_Conduct.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Expense_Reimbursement.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Performance_Reviews.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Processing Parking_Policy.pdf:
  Current chunks: 1
  Previous chunks: 1
  Changes detected: 1

Total changes detected: 20


---

## Step 5: Store Results

In [ ]:
if len(all_changes) > 0:
    if is_databricks():
        # Use custom Unity Catalog method
        inserted = db_handler.store_changes(all_changes, DETECTION_RUN_ID)
        print(f"✅ Stored {inserted} records in Unity Catalog")
        
        # Verify using Databricks handler
        verify_df = db_handler.verify_results(DETECTION_RUN_ID)
        
    else:
        # Use SQLite (original code)
        with sqlite3.connect(DB_PATH) as conn:
            inserted = ChangeDetectionQueries.store_change_detection_results(
                conn, all_changes, DETECTION_RUN_ID, use_spark=False
            )
        print(f"✅ Stored {inserted} records in SQLite")
        
        # Verify using SQLite
        with sqlite3.connect(DB_PATH) as conn:
            verify_df = pd.read_sql_query("""
                SELECT change_type, COUNT(*) as count
                FROM content_change_log WHERE detection_run_id = ?
                GROUP BY change_type
            """, conn, params=[DETECTION_RUN_ID])
    
    display(verify_df)
else:
    print("No changes detected to store")

2025-10-30 23:26:14.850 | INFO     | detection.database_queries:_store_changes_sqlite:278 - 💾 Stored 20/20 changes to database


✅ Stored 20 records in SQLite


change_type  count
0        new_content      7
1  unchanged_content     13

In [ ]:
if is_databricks():
    # Use Databricks handler to get all results
    all_results_df = db_handler.get_all_results(DETECTION_RUN_ID)
else:
    # Use SQLite (original code)
    with sqlite3.connect(DB_PATH) as conn:
        all_results_df = pd.read_sql_query("""
            SELECT *
            FROM content_change_log WHERE detection_run_id = ?
        """, conn, params=[DETECTION_RUN_ID])

display(all_results_df)

change_id                                   content_checksum  \
0           1  f3bbae03d26ce982636aaa5552ad07e8735c7dd28f36d3...   
1           2  b54c22ec37e28c72f242c5c96ba13a0d582cf14ad4bba7...   
2           3  618294d1ec9e3f03260d3e7015c33734c3db3b8ffa543a...   
3           4  21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...   
4           5  b83c5c009d7ae9abaaf977909eada2be578234e052751c...   
5           6  525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...   
6           7  7317c6901c9282b3c1adbab4f0b5937f75d431737d2c49...   
7           8  c0a9947d707ea77c52566453c69cc6aef7cb3f07bcb742...   
8           9  72cb4a8e3750676f31506c2a95bc333bfaef0f916f020b...   
9          10  f728f4272c384ce30152a0bcf38e83d2bd3869f5a5be92...   
10         11  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...   
11         12  691b8506507e69ae90edc609bb29735aa050604d10e575...   
12         13  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...   
13         14  320f9f24dbf9c03edfdba60c2c0e973eff5c0932dc8183...   
14         15  e4e657579f05c0b56e0cf9d2c3c60bcbfae818ac84cedd...   
15         16  32cbadc16b6c3e6976b4158e3ffd945855495cb3ccc17d...   
16         17  a63a029055575b7aaedfa8d55dad9b859305a101b1f5e7...   
17         18  21c3224cf3de0229bb0953c537dfe703be7752c5a3513e...   
18         19  5344d68abbaa53f9ce78175e24ce7282042d98399bc4a8...   
19         20  9638205ff31b28380b3165acf62ba48de76e2cf45fadda...   

                                    previous_checksum  \
0                                                None   
1   b54c22ec37e28c72f242c5c96ba13a0d582cf14ad4bba7...   
2                                                None   
3   21f1d25d4f2dc7ce53d72cc050c32738362e1e12536d0b...   
4                                                None   
5   525a301d7750b939ac2adf0e9f3690770b81277fd5dcc8...   
6                                                None   
7   c0a9947d707ea77c52566453c69cc6aef7cb3f07bcb742...   
8                                                None   
9                                                None   
10  370cd156d5ae42b3bb93fe51096b05d8dc2aee7358aaee...   
11                                               None   
12  26b8fffa2bb44d9ed01d6878fa75f7efc7c3d6355478e3...   
13  320f9f24dbf9c03edfdba60c2c0e973eff5c0932dc8183...   
14  e4e657579f05c0b56e0cf9d2c3c60bcbfae818ac84cedd...   
15  32cbadc16b6c3e6976b4158e3ffd945855495cb3ccc17d...   
16  a63a029055575b7aaedfa8d55dad9b859305a101b1f5e7...   
17  21c3224cf3de0229bb0953c537dfe703be7752c5a3513e...   
18  5344d68abbaa53f9ce78175e24ce7282042d98399bc4a8...   
19  9638205ff31b28380b3165acf62ba48de76e2cf45fadda...   

                    file_name  requires_faq_regeneration        change_type  \
0       Employee_Handbook.pdf                          1        new_content   
1       Employee_Handbook.pdf                          0  unchanged_content   
2      IT_Security_Policy.pdf                          1        new_content   
3      IT_Security_Policy.pdf                          0  unchanged_content   
4      Remote_Work_Policy.pdf                          1        new_content   
5      Remote_Work_Policy.pdf                          0  unchanged_content   
6           Travel_Policy.pdf                          1        new_content   
7           Travel_Policy.pdf                          0  unchanged_content   
8          Parental_Leave.pdf                          1        new_content   
9        Health_Insurance.pdf                          1        new_content   
10       Health_Insurance.pdf                          0  unchanged_content   
11           Email_Policy.pdf                          1        new_content   
12           Email_Policy.pdf                          0  unchanged_content   
13       Equipment_Return.pdf                          0  unchanged_content   
14         Benefits_Guide.pdf                          0  unchanged_content   
15     Payroll_Processing.pdf                          0  unchanged_content   
16        Code_of_Conduct.pdf     

In [ ]:
# Access diff_data from the results (reuse the variable from previous cell)
try:
    if 'all_results_df' in locals() and len(all_results_df) > 0:
        all_results_df.loc[0, 'diff_data']
    else:
        print("No results available to display diff_data")
except Exception as e:
    print(f"Error accessing diff_data: {e}")

---

## Summary

In [ ]:
new = [c for c in all_changes if c.change_type == ChangeType.NEW_CONTENT]
modified = [c for c in all_changes if c.change_type == ChangeType.MODIFIED_CONTENT]
deleted = [c for c in all_changes if c.change_type == ChangeType.DELETED_CONTENT]

print("="*80)
print("✅ DETECTION COMPLETE")
print("="*80)
print(f"Environment: {'Databricks Unity Catalog' if is_databricks() else 'Local SQLite'}")
print(f"Run ID: {DETECTION_RUN_ID}")
print(f"Changes: {len(all_changes)} (NEW: {len(new)}, MOD: {len(modified)}, DEL: {len(deleted)})")

if is_databricks():
    print(f"Results stored in: {CATALOG}.{SCHEMA}.content_change_log")
else:
    print(f"Results stored in: {DB_PATH}")

print("="*80)

if is_databricks():
    print("\n🎯 Next Steps for Databricks:")
    print("   1. Results are now stored in Unity Catalog Delta tables")
    print("   2. Use Databricks SQL or notebooks to query results")
    print("   3. Set up scheduled jobs to run change detection automatically")
    print("   4. Consider setting up Delta Live Tables for real-time processing")